# 001.01 // data generation

[take photographs of playing cards to generate a dataset of card pairings against randomly-selected backgrounds from the Oxford Describable Textures Dataset]

In [ ]:
import os
import shutil
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
from glob import glob
import cv2

import deck

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
# set up data directory

data_dir = '../data/card_data/'

os.mkdir('../data/')
os.mkdir(data_dir)
os.mkdir(f'{data_dir}photos/')
os.mkdir(f'{data_dir}cards/')
os.mkdir('../data/images/')
os.mkdir('../data/labels/')

## photograph cards

In [ ]:
# take photos ace through king one suit at a time on a uniform background in contrast with the card
# make sure suit order below is same as photo order
# put photos in dir: ../data/card_data/photos/

card_names = []

for suit in ['c', 'd', 'h', 's']:
    for rank in range(1, 14):
        card_names.append(f'{suit}{rank:02}')
        
len(card_names)

## extraction

In [ ]:
# measure card specs in millimeters and record below

cardW=63
cardH=87
cornerXmin=1 # distance from edge of card to beginning of corner suit (horizontal)
cornerXmax=8.95 # distance from edge of card to end of corner suit (horizontal)
cornerYmin=3 # distance from edge of card to beginning of corner suit (vertical)
cornerYmax=23 # distance from edge of card to end of corner suit (vertical)

In [ ]:
photo_dir = f'{data_dir}photos/'
card_dir = f'{data_dir}cards/'

cards_pck_fn=data_dir+"cards.pkl"
backgrounds_pck_fn=data_dir+"backgrounds.pkl"

card_suits=['c', 'd', 'h', 's']
card_ranks=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13']

zoom=4
cardW*=zoom
cardH*=zoom
cornerXmin=int(cornerXmin*zoom)
cornerXmax=int(cornerXmax*zoom)
cornerYmin=int(cornerYmin*zoom)
cornerYmax=int(cornerYmax*zoom)

refCard=np.array([[0,0],[cardW,0],[cardW,cardH],[0,cardH]],dtype=np.float32)
refCardRot=np.array([[cardW,0],[cardW,cardH],[0,cardH],[0,0]],dtype=np.float32)
refCornerHL=np.array([[cornerXmin,cornerYmin],[cornerXmax,cornerYmin],[cornerXmax,cornerYmax],[cornerXmin,cornerYmax]],dtype=np.float32)
refCornerLR=np.array([[cardW-cornerXmax,cardH-cornerYmax],[cardW-cornerXmin,cardH-cornerYmax],[cardW-cornerXmin,cardH-cornerYmin],[cardW-cornerXmax,cardH-cornerYmin]],dtype=np.float32)
refCorners=np.array([refCornerHL,refCornerLR])

In [ ]:
# create mask for extraction

bord_size=2
alphamask=np.ones((cardH,cardW),dtype=np.uint8)*255
cv2.rectangle(alphamask,(0,0),(cardW-1,cardH-1),0,bord_size)
cv2.line(alphamask,(bord_size*3,0),(0,bord_size*3),0,bord_size)
cv2.line(alphamask,(cardW-bord_size*3,0),(cardW,bord_size*3),0,bord_size)
cv2.line(alphamask,(0,cardH-bord_size*3),(bord_size*3,cardH),0,bord_size)
cv2.line(alphamask,(cardW-bord_size*3,cardH),(cardW,cardH-bord_size*3),0,bord_size)
plt.figure(figsize=(10,10))
plt.imshow(alphamask);

In [ ]:
# extract cards from photos
# make sure all processed properly

i = 0
filenames = sorted([f.split('.jpeg')[0] for f in os.listdir(photo_dir) if '.jpeg' in f])

for f in filenames:
    debug=False
    img=cv2.imread(f'{photo_dir}{f}.jpeg')
    valid,card=deck.extract_card(img, alphamask, f'{card_dir}{card_names[i]}.png', debug=debug)
    i += 1
    if valid:
        deck.display_img(card)

In [ ]:
# test a few cards at random to make sure corner suit is completely in bounding box

imgs_fns=glob(card_dir+"/*.png")
img_fn=random.choice(imgs_fns)
deck.display_img(cv2.imread(img_fn,cv2.IMREAD_UNCHANGED),polygons=[refCornerHL,refCornerLR])

In [ ]:
# test a few cards at random to make sure convex hull is properly identified

debug="no"
img_fn=random.choice(imgs_fns)
img=cv2.imread(img_fn,cv2.IMREAD_UNCHANGED)
hullHL=deck.findHull(img,refCornerHL,debug=debug)
hullLR=deck.findHull(img,refCornerLR,debug=debug)
deck.display_img(img,[refCornerHL,refCornerLR,hullHL,hullLR])

## augmentation

(optional) before finalizing set of cards, use the augmentation notebook to create additional versions of a card (adjust brightness, color, contrast, etc)

In [ ]:
# post-augmentation, run cell to finalize set of cards

cards={}
for suit in card_suits:
    for rank in card_ranks:
        card_name=suit+rank       
        cd=os.path.join(card_dir,card_name)
        if not os.path.isdir(cd):
            print(f"!!! {cd} does not exist !!!")
            continue
        cards[card_name]=[]
        for f in glob(f'{cd}/*.png'):
            img=cv2.imread(f,cv2.IMREAD_UNCHANGED)
            hullHL=deck.findHull(img,refCornerHL,debug="no") 
            if hullHL is None: 
                print(f"File {f} not used.")
                continue
            hullLR=deck.findHull(img,refCornerLR,debug="no") 
            if hullLR is None: 
                print(f"File {f} not used.")
                continue
            img=cv2.cvtColor(img,cv2.COLOR_BGRA2RGBA)
            cards[card_name].append((img,hullHL,hullLR))



print("saved as:",cards_pck_fn)
pickle.dump(cards,open(cards_pck_fn,'wb'))

In [ ]:
cards = deck.Cards()

In [ ]:
# test a few cards at random to make sure set is properly loaded

_=cards.get_random(display=True)

## background images

In [ ]:
# download the Oxford Describable Textures Dataset (DTD)

!wget https://www.robots.ox.ac.uk/~vgg/data/dtd/download/dtd-r1.0.1.tar.gz -P ../data/card_data/

In [ ]:
shutil.unpack_archive(f'{data_dir}dtd-r1.0.1.tar.gz', data_dir)

In [ ]:
dtd_dir=f'{data_dir}dtd/images/'
bg_images=[]
for subdir in glob(dtd_dir+"/*"):
    for f in glob(subdir+"/*.jpg"):
        bg_images.append(mpimg.imread(f))
print("saved as:",backgrounds_pck_fn)
pickle.dump(bg_images,open(backgrounds_pck_fn,'wb'))

In [ ]:
!rm -r ../data/card_data/dtd
!rm ../data/card_data/dtd-r1.0.1.tar.gz

In [ ]:
backgrounds = deck.Backgrounds()

In [ ]:
# test a few backgrounds at random to make sure dataset is properly loaded

_=backgrounds.get_random(display=True)

In [ ]:
# test sample images representative of final datatset at random

bg=backgrounds.get_random()
img1,card_val1,hulla1,hullb1=cards.get_random()
img2,card_val2,hulla2,hullb2=cards.get_random()

newimg=deck.Scene(bg,img1,card_val1,hulla1,hullb1,img2,card_val2,hulla2,hullb2)
newimg.display()

In [ ]:
# generate dataset

cards_to_generate=51200
save_dir="../data/images"

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

for i in tqdm(range(cards_to_generate)):
    bg=backgrounds.get_random()
    img1,card_val1,hulla1,hullb1=cards.get_random()
    img2,card_val2,hulla2,hullb2=cards.get_random()
    
    newimg=deck.Scene(bg,img1,card_val1,hulla1,hullb1,img2,card_val2,hulla2,hullb2)
    newimg.write_files(save_dir)

## augmentation

(optional) use the augmentation notebook to convert images to grayscale